Caricamento dataset

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("/content/dataset_LUMINAL_A_B(in).csv")
# Separare features (gene expression) e labels (Luminal A/B)
X = dataset.drop('l', axis=1).values  # Le espressioni geniche
y = dataset['l'].values  # Le etichette (Luminal A/B)

dataset.sample()
X[0]
y[0]
#in X ci sono i nodi(righe) con le features/gene expressions(colonne)
#in y ci sono le ground truths/labels per ogni nodo(riga)

'Luminal A    '

Preprocessing dati e labels

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA


encoder = LabelEncoder()
y = encoder.fit_transform(y)
#Le etichette vengono convertite in formato numerico (0,1) per poter essere utilizzate in modelli di machine learning.

GNN: Creo il grafo, con gli edges, labels, features, costruisco GNN e la addestro e testo


In [47]:
import torch
#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from sklearn.metrics import accuracy_score
import numpy as np

# 1.Costruisco il Grafo con la Pearson Correlation

# Calcolare la matrice di correlazione di Pearson tra i vettori di espressione genica dei pazienti (nodi)
correlation_matrix = np.corrcoef(X)  # Correlazione tra righe (pazienti)
# Stampa delle dimensioni del dataset e della matrice di correlazione
print(f"Dimensioni del dataset: {X.shape}")
print(f"Dimensioni della matrice di correlazione: {correlation_matrix.shape}")
# Creiamo un edge solo se la correlazione tra due pazienti è alta
threshold = 0.5
adjacency_matrix = (correlation_matrix > threshold).astype(int)
# converto la matrice di adiacenza in edge_index (formato richiesto da PyTorch Geometric dove la prima riga contiene gli indici dei nodi di partenza degli edge, la seconda riga contiene gli indici dei nodi di arrivo degli edge.)
edge_index, _ = dense_to_sparse(torch.tensor(adjacency_matrix))

print(correlation_matrix)
print(adjacency_matrix)
print(edge_index)

#2. Creo l'oggetto Data per PyTorch Geometric
data = Data(
    x=torch.tensor(X, dtype=torch.float32),  # Feature dei nodi (gene expression)
    edge_index=edge_index,  # Gli edge costruiti dalla Pearson correlation
    y=torch.tensor(y, dtype=torch.long)  # Le etichette (Luminal A/B)
)




Dimensioni del dataset: (100, 1022)
Dimensioni della matrice di correlazione: (100, 100)
[[1.         0.6540297  0.2532472  ... 0.18841422 0.43839602 0.20150986]
 [0.6540297  1.         0.78824131 ... 0.75353536 0.84955123 0.74069147]
 [0.2532472  0.78824131 1.         ... 0.99686232 0.76484103 0.99534994]
 ...
 [0.18841422 0.75353536 0.99686232 ... 1.         0.75260984 0.99711608]
 [0.43839602 0.84955123 0.76484103 ... 0.75260984 1.         0.74845195]
 [0.20150986 0.74069147 0.99534994 ... 0.99711608 0.74845195 1.        ]]
[[1 1 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]
tensor([[ 0,  0,  0,  ..., 99, 99, 99],
        [ 0,  1,  4,  ..., 97, 98, 99]])


In [48]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# 3. Definisco il Modello GCN
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Parametri del Modello
input_dim = X.shape[1]  # Numero di feature (espressioni geniche)
hidden_dim = 64  # Numero di neuroni nel layer nascosto
output_dim = 2   # Numero di classi (Luminal A / Luminal B)

# Inizializziamo il modello
model = GCN(input_dim, hidden_dim, output_dim)

# 4. Definire l'ottimizzatore e la funzione di loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# 5. Suddividere i dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_mask = torch.tensor([i for i in range(len(X_train))], dtype=torch.long)
test_mask = torch.tensor([i + len(X_train) for i in range(len(X_test))], dtype=torch.long)

# Creare i tensor per train e test
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 6. Addestrare il modello GNN
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(data.x, data.edge_index)

    # Calcolare la loss per il training
    loss = criterion(out[train_mask], data.y[train_mask])

    # Backward pass
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 7. Valutare il modello sul test set
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    # Calcolare l'accuratezza
    train_acc = accuracy_score(y_train, pred[:len(y_train)].cpu().numpy())
    test_acc = accuracy_score(y_test, pred[len(y_train):].cpu().numpy())

print(f'Train Accuracy: {train_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')


Epoch [10/100], Loss: 2116887.5000
Epoch [20/100], Loss: 978955.0000
Epoch [30/100], Loss: 568127.1250
Epoch [40/100], Loss: 76337.3906
Epoch [50/100], Loss: 551377.4375
Epoch [60/100], Loss: 189083.1875
Epoch [70/100], Loss: 84465.4609
Epoch [80/100], Loss: 134491.2188
Epoch [90/100], Loss: 137690.4219
Epoch [100/100], Loss: 65713.0391
Train Accuracy: 0.5125
Test Accuracy: 0.4500


Confronto con MLP

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 1. Definire il modello MLP
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=0.5)  # Dropout per regolarizzare il modello

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.log_softmax(x, dim=1)

# Parametri del modello
input_dim = X.shape[1]  # Numero di feature (espressioni geniche)
hidden_dim = 64  # Numero di neuroni nei layer nascosti
output_dim = 2   # Numero di classi (Luminal A / Luminal B)

# Inizializzare il modello MLP
mlp_model = MLP(input_dim, hidden_dim, output_dim)

# 2. Definire l'ottimizzatore e la funzione di loss
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.01, weight_decay=5e-4)
mlp_criterion = nn.CrossEntropyLoss()

# 3. Suddividere i dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertire in tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 4. Addestrare l'MLP
num_epochs = 100
for epoch in range(num_epochs):
    mlp_model.train()
    mlp_optimizer.zero_grad()

    # Forward pass
    out = mlp_model(X_train_tensor)

    # Calcolare la loss per il training
    loss = mlp_criterion(out, y_train_tensor)

    # Backward pass
    loss.backward()
    mlp_optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 5. Valutare l'MLP sul test set
mlp_model.eval()
with torch.no_grad():
    out = mlp_model(X_test_tensor)
    pred = out.argmax(dim=1)

    # Calcolare l'accuratezza
    test_acc = accuracy_score(y_test, pred)

print(f'Test Accuracy (MLP): {test_acc:.4f}')


Epoch [10/100], Loss: 26042.6504
Epoch [20/100], Loss: 88.6739
Epoch [30/100], Loss: 0.6830
Epoch [40/100], Loss: 0.6844
Epoch [50/100], Loss: 0.6922
Epoch [60/100], Loss: 0.6911
Epoch [70/100], Loss: 0.6834
Epoch [80/100], Loss: 0.6835
Epoch [90/100], Loss: 0.6835
Epoch [100/100], Loss: 0.6919
Test Accuracy (MLP): 0.4000
